In [ ]:
# !pip install hdbscan
# !pip install umap-learn

In [ ]:
import re
# import umap
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn.manifold as manifold

# import hdbscan
from nltk.corpus import stopwords
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from sklearn.metrics import silhouette_score

from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import HoverTool, ColumnDataSource, CategoricalColorMapper
from bokeh.palettes import Spectral10
output_notebook()

In [ ]:
!gdown --id 1o31_gh2MLtfu0gpXkhyyZBoAdnKbvyQr #title.basics.tsv
!gdown --id 1XruIRqJFTB7lTrc2aAIjWfY7AbxdgKzj #title.ratings.tsv
!gdown --id 1Eu12S_gsiliEzLMbXIi3esZ2g4wOmQuN #title.principals.tsv
!gdown --id 13LF4mnwAQU3HN3wgXGrgc6dNuZa5sGOU #title.akas.tsv

Downloading...
From: https://drive.google.com/uc?id=1o31_gh2MLtfu0gpXkhyyZBoAdnKbvyQr
To: /content/title.basics.tsv
657MB [00:12, 53.9MB/s]
Downloading...
From: https://drive.google.com/uc?id=1XruIRqJFTB7lTrc2aAIjWfY7AbxdgKzj
To: /content/title.ratings.tsv
19.3MB [00:00, 34.2MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Eu12S_gsiliEzLMbXIi3esZ2g4wOmQuN
To: /content/title.principals.tsv
1.92GB [00:26, 71.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=13LF4mnwAQU3HN3wgXGrgc6dNuZa5sGOU
To: /content/title.akas.tsv
1.26GB [00:20, 60.5MB/s]


In [ ]:
dfMovies1 = pd.read_csv("/content/title.basics.tsv", sep='\t')
dfMovies2 = pd.read_csv("/content/title.ratings.tsv", sep='\t')
dfMovies3 = pd.read_csv("/content/title.principals.tsv", sep='\t')
dfMovies4 = pd.read_csv("/content/title.akas.tsv", sep='\t')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
dfMovies1.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [ ]:
dfMovies1.shape

(7697093, 9)

In [ ]:
dfMovies2.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1688
1,tt0000002,6.0,209
2,tt0000003,6.5,1431
3,tt0000004,6.1,122
4,tt0000005,6.1,2229


In [ ]:
dfMovies2.shape

(1131280, 3)

In [ ]:
dfMovies3.head()

,tconst,ordering,nconst,category,job,characters
0,tt0000001,1,nm1588970,self,\N,"[""Self""]"
1,tt0000001,2,nm0005690,director,\N,\N
2,tt0000001,3,nm0374658,cinematographer,director of photography,\N
3,tt0000002,1,nm0721526,director,\N,\N
4,tt0000002,2,nm1335271,composer,\N,\N


In [ ]:
dfMovies3.shape

(43512883, 6)

In [ ]:
dfMovies4.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,\N,imdbDisplay,\N,0
1,tt0000001,2,Carmencita,DE,\N,\N,literal title,0
2,tt0000001,3,Carmencita - spanyol tánc,HU,\N,imdbDisplay,\N,0
3,tt0000001,4,Καρμενσίτα,GR,\N,imdbDisplay,\N,0
4,tt0000001,5,Карменсита,RU,\N,imdbDisplay,\N,0


In [ ]:
dfMovies4.shape

(25485617, 8)

In [ ]:
dfMovies4['region'].unique()

array(['UA', 'DE', 'HU', 'GR', 'RU', 'US', '\\N', 'JP', 'FR', 'RO', 'GB',
       'PT', 'RS', 'ES', 'UY', 'IT', 'AR', 'FI', 'PL', 'BR', 'DK', 'TR',
       'XWW', 'XEU', 'SK', 'CZ', 'SE', 'MX', 'NO', 'XYU', 'AT', 'VE',
       'CSHH', 'SI', 'AU', 'TW', 'LT', 'IN', 'CA', 'NL', 'CO', 'IR', 'BG',
       'BE', 'DZ', 'CH', 'NZ', 'BF', 'XWG', 'VN', 'XSA', 'SUHH', 'EE',
       'IS', 'HR', 'DDDE', 'HK', 'XKO', 'CL', 'PE', 'EG', 'GE', 'BY',
       'BA', 'PA', 'CN', 'TJ', 'XSI', 'YUCS', 'IE', 'IL', 'LV', 'CU',
       'AL', 'BO', 'ID', 'ZA', 'KR', 'UZ', 'MY', 'SG', 'BUMM', 'XPI',
       'BJ', 'PR', 'CM', 'AZ', 'TH', 'DO', 'EC', 'MA', 'PH', 'GL', 'MN',
       'LI', 'LU', 'PY', 'MZ', 'GT', 'BM', 'JM', 'MD', 'CR', 'LB', 'IQ',
       'TM', 'MK', 'TN', 'NG', 'HT', 'AM', 'SN', 'GH', 'CI', 'JO', 'KG',
       'PK', 'LK', 'KZ', 'NE', 'GN', 'VDVN', 'TD', 'SO', 'SD', 'MC', 'CG',
       'TT', 'GA', 'BS', 'LY', 'SY', 'AO', 'KH', 'SV', 'MR', 'AF', 'MG',
       'ML', 'GY', 'CY', 'ET', 'GU', 'XAS', 'SR', 'MT', 'TG'

In [ ]:
dfMovies4 = dfMovies4[dfMovies4['region'] == 'US'][['titleId', 'title']].rename(columns={'titleId': 'tconst'})
dfMovies4.head(10)

,tconst,title
5,tt0000001,Carmencita
14,tt0000002,The Clown and His Dogs
35,tt0000005,Blacksmithing Scene
39,tt0000005,Blacksmith Scene #1
40,tt0000005,Blacksmithing
43,tt0000005,Blacksmith Scene
46,tt0000006,Chinese Opium Den
50,tt0000007,Corbett and Courtney Before the Kinetograph
54,tt0000007,The Corbett-Courtney Fight
55,tt0000007,Jim Corbett vs. Peter Courtney


In [ ]:
dfMovieTitles = pd.merge(dfMovies1, dfMovies4[['tconst', 'title']], on=['tconst'], how='inner')
dfMovieTitles.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,title
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short",Carmencita
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short",The Clown and His Dogs
2,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short",Blacksmithing Scene
3,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short",Blacksmith Scene #1
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short",Blacksmithing


In [ ]:
dfMovieTitles = dfMovieTitles.drop_duplicates(['tconst'])
dfMovieTitles.head(10)

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,title
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short",Carmencita
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short",The Clown and His Dogs
2,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short",Blacksmithing Scene
6,tt0000006,short,Chinese Opium Den,Chinese Opium Den,0,1894,\N,1,Short,Chinese Opium Den
7,tt0000007,short,Corbett and Courtney Before the Kinetograph,Corbett and Courtney Before the Kinetograph,0,1894,\N,1,"Short,Sport",Corbett and Courtney Before the Kinetograph
10,tt0000008,short,Edison Kinetoscopic Record of a Sneeze,Edison Kinetoscopic Record of a Sneeze,0,1894,\N,1,"Documentary,Short","Edison Kinetoscopic Record of a Sneeze, Januar..."
12,tt0000009,short,Miss Jerry,Miss Jerry,0,1894,\N,40,"Romance,Short",Miss Jerry
13,tt0000010,short,Leaving the Factory,La sortie de l'usine Lumière à Lyon,0,1895,\N,1,"Documentary,Short",Leaving the Factory
17,tt0000012,short,The Arrival of a Train,L'arrivée d'un train à La Ciotat,0,1896,\N,1,"Documentary,Short",The Arrival of a Train
19,tt0000013,short,The Photographical Congress Arrives in Lyon,Neuville-sur-Saône: Débarquement du congrès de...,0,1895,\N,1,"Documentary,Short",The Photographical Congress Arrives in Lyon


In [ ]:
print(dfMovies3['category'].unique())

['self' 'director' 'cinematographer' 'composer' 'producer' 'editor'
 'actor' 'actress' 'writer' 'production_designer' 'archive_footage'
 'archive_sound']


In [ ]:
dfCastActor = dfMovies3[dfMovies3['category'].str.contains('actress|actor', na=False)][['tconst','nconst']].rename(columns={"nconst": "ActorNum"})
dfCastActor['ActorNum'] = dfCastActor['ActorNum'].str.replace(r'\D', '').astype(int)
dfCastDirector = dfMovies3[dfMovies3['category'].str.contains('director', na=False)][['tconst','nconst']].rename(columns={"nconst": "DirectorNum"})
dfCastDirector['DirectorNum'] = dfCastDirector['DirectorNum'].str.replace(r'\D', '').astype(int)
dfCastWriter = dfMovies3[dfMovies3['category'].str.contains('writer', na=False)][['tconst','nconst']].rename(columns={"nconst": "WriterNum"})
dfCastWriter['WriterNum'] = dfCastWriter['WriterNum'].str.replace(r'\D', '').astype(int)

In [ ]:
dfCastActor = dfCastActor.groupby(['tconst']).sum().reset_index()
dfCastDirector = dfCastDirector.groupby(['tconst']).sum().reset_index()
dfCastWriter = dfCastWriter.groupby(['tconst']).sum().reset_index()

In [ ]:
dfCastActor.shape

(4315940, 2)

In [ ]:
dfCastDirector.shape

(3921612, 2)

In [ ]:
dfCastDirector.head()

,tconst,DirectorNum
0,tt0000001,5690
1,tt0000002,721526
2,tt0000003,721526
3,tt0000004,721526
4,tt0000005,5690


In [ ]:
dfCastWriter.shape

(2748903, 2)

In [ ]:
dfCast = pd.merge(pd.merge(dfCastDirector, dfCastActor, on=['tconst'], how='outer'), dfCastWriter, on=['tconst'], how='outer')
dfCast.head()

,tconst,DirectorNum,ActorNum,WriterNum
0,tt0000001,5690.0,NaN,NaN
1,tt0000002,721526.0,NaN,NaN
2,tt0000003,721526.0,NaN,NaN
3,tt0000004,721526.0,NaN,NaN
4,tt0000005,5690.0,1096524.0,NaN


In [ ]:
dfCast.shape

(5454370, 4)

In [ ]:
dfCast['crewNum']= dfCast.iloc[:,-3:].sum(axis=1).astype(int)
dfCast.head()

,tconst,DirectorNum,ActorNum,WriterNum,crewNum
0,tt0000001,5690.0,NaN,NaN,5690
1,tt0000002,721526.0,NaN,NaN,721526
2,tt0000003,721526.0,NaN,NaN,721526
3,tt0000004,721526.0,NaN,NaN,721526
4,tt0000005,5690.0,1096524.0,NaN,1102214


In [ ]:
dfMovies = pd.merge(dfMovieTitles, dfMovies2[['tconst', 'averageRating',	'numVotes']], on=['tconst'], how='left')
dfMovies = pd.merge(dfMovies, dfCast[['tconst', 'crewNum']], on=['tconst'], how='inner')
dfMovies.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,title,averageRating,numVotes,crewNum
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short",Carmencita,5.7,1688.0,5690
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short",The Clown and His Dogs,6.0,209.0,721526
2,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short",Blacksmithing Scene,6.1,2229.0,1102214
3,tt0000006,short,Chinese Opium Den,Chinese Opium Den,0,1894,\N,1,Short,Chinese Opium Den,5.3,123.0,5690
4,tt0000007,short,Corbett and Courtney Before the Kinetograph,Corbett and Courtney Before the Kinetograph,0,1894,\N,1,"Short,Sport",Corbett and Courtney Before the Kinetograph,5.4,684.0,743458


In [ ]:
del dfMovies['primaryTitle']
del dfMovies['originalTitle']
dfMovies = dfMovies.rename(columns={"title": "originalTitle"})

In [ ]:
dfMovies.shape

(973371, 11)

In [ ]:
dfMovies['titleType'].unique()

array(['short', 'movie', 'tvSeries', 'tvMovie', 'tvEpisode', 'tvShort',
       'tvMiniSeries', 'video', 'tvSpecial', 'videoGame'], dtype=object)

In [ ]:
dfMovies = dfMovies[dfMovies['titleType'].str.contains('movie|tvMovie', na=False)]

In [ ]:
dfMovies.shape

(240181, 11)

In [ ]:
dfMovies[dfMovies['endYear'].str.contains(r"\\N", na=False)][['tconst','endYear']].shape

(240181, 2)

In [ ]:
dfMovies[dfMovies['runtimeMinutes'].str.contains(r"\\N", na=False)][['tconst','runtimeMinutes']].shape

(78535, 2)

In [ ]:
dfMovies[dfMovies['genres'].str.contains(r'\\N', na=False)][['tconst','genres']].shape

(8097, 2)

In [ ]:
dfMovies[dfMovies['titleType'].str.contains(r"\\N", na=False)][['tconst','titleType']].shape

(0, 2)

In [ ]:
dfMovies[dfMovies['isAdult'].str.contains(r"\\N", na=False)][['tconst','isAdult']].shape

(0, 2)

In [ ]:
dfMovies[dfMovies['originalTitle'].str.contains(r"\\N", na=False)][['tconst','originalTitle']].shape

(0, 2)

In [ ]:
dfMovies[dfMovies['startYear'].str.contains(r"\\N", na=False)][['tconst','startYear']].shape

(43787, 2)

In [ ]:
dfMovies[dfMovies['genres'].str.contains(r"\\N", na=False)][['tconst','genres']].shape

(8097, 2)

In [ ]:
dfMovies = dfMovies[dfMovies['genres'] != r"\N"]

In [ ]:
dfMovies.shape

(232084, 11)

In [ ]:
dfMovies = dfMovies[dfMovies['startYear'] != r"\N"]
dfMovies.shape

(189219, 11)

In [ ]:
dfMovies = dfMovies[dfMovies['isAdult'] != 1] #No Adult content in recommender
del dfMovies['isAdult']
del dfMovies['endYear']
del dfMovies['runtimeMinutes']

In [ ]:
dfMovies.shape

(183578, 8)

In [ ]:
dfMovies.isna().sum()

tconst               0
titleType            0
startYear            0
genres               0
originalTitle        0
averageRating    58309
numVotes         58309
crewNum              0
dtype: int64

In [ ]:
dfMovies['averageRating'] = dfMovies['averageRating'].fillna(0)
dfMovies['numVotes'] = dfMovies['numVotes'].fillna(0)

In [ ]:
dfMovies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 183578 entries, 211 to 973364
Data columns (total 8 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   tconst         183578 non-null  object 
 1   titleType      183578 non-null  object 
 2   startYear      183578 non-null  object 
 3   genres         183578 non-null  object 
 4   originalTitle  183578 non-null  object 
 5   averageRating  183578 non-null  float64
 6   numVotes       183578 non-null  float64
 7   crewNum        183578 non-null  int64  
dtypes: float64(2), int64(1), object(5)
memory usage: 12.6+ MB


In [ ]:
dfMovies = dfMovies.dropna()

In [ ]:
dfMovies.shape

(183578, 8)

In [ ]:
dfMovies.reset_index(drop=True)

,tconst,titleType,startYear,genres,originalTitle,averageRating,numVotes,crewNum
0,tt0000630,movie,1908,Drama,Hamlet,3.2,11.0,768415
1,tt0000679,movie,1908,"Adventure,Fantasy",The Fairylogue and Radio-Plays,5.2,34.0,4951455
2,tt0000886,movie,1910,Drama,"Hamlet, Prince of Denmark",5.0,23.0,710351
3,tt0001285,movie,1909,"Biography,Drama,Family",Forty Years in the Land of the Midian,5.3,28.0,3686464
4,tt0001498,movie,1911,War,The Battle of Trafalgar,7.7,6.0,4114934
...,...,...,...,...,...,...,...,...
183573,tt9915546,tvMovie,2017,Comedy,Hot Explicit Ladies,0.0,0.0,10187868
183574,tt9915998,tvMovie,2016,Music,"Adidas x Pusha T at Art Basel, Miami",0.0,0.0,2334213
183575,tt9916190,movie,2020,"Action,Adventure,Thriller",Safeguard,3.4,139.0,18735420
183576,tt9916362,movie,2020,"Adventure,Drama,History",Coven,6.2,388.0,20183766


In [ ]:
dfMovies['title'] = dfMovies['originalTitle'].str.lower()
dfMovies['titleID'] = dfMovies['tconst']

In [ ]:
dfMovies['tconst'] = dfMovies['tconst'].str.replace(r'\D', '').astype(int)
dfMovies['startYear'] = dfMovies['startYear'].astype(int)
dfMovies['numVotes'] = dfMovies['numVotes'].astype(int)

In [ ]:
def convertToInt(row):
  sum = 0
  for i in row:
      sum+=ord(i)
  return sum

In [ ]:
#Converting titleType, originalTitle and genres to int
dfMovies['genres'] = dfMovies['genres'].apply(lambda x: convertToInt(x))
dfMovies['titleType'] = dfMovies['titleType'].apply(lambda x: convertToInt(x))
dfMovies['originalTitle'] = dfMovies['originalTitle'].apply(lambda x: convertToInt(x))
dfMovies.rename(columns={"genres":"genreNum", "originalTitle":"titleNum"})

,tconst,titleType,startYear,genreNum,titleNum,averageRating,numVotes,crewNum,title,titleID
211,630,544,1908,485,603,3.2,11,768415,hamlet,tt0000630
240,679,544,1908,1712,2800,5.2,34,4951455,the fairylogue and radio-plays,tt0000679
390,886,544,1910,485,2271,5.0,23,710351,"hamlet, prince of denmark",tt0000886
657,1285,544,1909,2116,3319,5.3,28,3686464,forty years in the land of the midian,tt0001285
804,1498,544,1911,298,2118,7.7,6,4114934,the battle of trafalgar,tt0001498
...,...,...,...,...,...,...,...,...,...,...
973339,9915546,746,2017,609,1791,0.0,0,10187868,hot explicit ladies,tt9915546
973353,9915998,746,2016,513,3055,0.0,0,2334213,"adidas x pusha t at art basel, miami",tt9915998
973356,9916190,544,2020,2474,914,3.4,139,18735420,safeguard,tt9916190
973362,9916362,544,2020,2269,507,6.2,388,20183766,coven,tt9916362


In [ ]:
dfMovies.head()

,tconst,titleType,startYear,genres,originalTitle,averageRating,numVotes,crewNum,title,titleID
211,630,544,1908,485,603,3.2,11,768415,hamlet,tt0000630
240,679,544,1908,1712,2800,5.2,34,4951455,the fairylogue and radio-plays,tt0000679
390,886,544,1910,485,2271,5.0,23,710351,"hamlet, prince of denmark",tt0000886
657,1285,544,1909,2116,3319,5.3,28,3686464,forty years in the land of the midian,tt0001285
804,1498,544,1911,298,2118,7.7,6,4114934,the battle of trafalgar,tt0001498


In [ ]:
dfMovies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 183578 entries, 211 to 973364
Data columns (total 10 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   tconst         183578 non-null  int64  
 1   titleType      183578 non-null  int64  
 2   startYear      183578 non-null  int64  
 3   genres         183578 non-null  int64  
 4   originalTitle  183578 non-null  int64  
 5   averageRating  183578 non-null  float64
 6   numVotes       183578 non-null  int64  
 7   crewNum        183578 non-null  int64  
 8   title          183578 non-null  object 
 9   titleID        183578 non-null  object 
dtypes: float64(1), int64(7), object(2)
memory usage: 15.4+ MB


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
dfMovies.to_csv('/content/drive/My Drive/Big Data Project/data/IMDBMovies.csv', index=False)